In [1]:
import keras
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/refs/heads/master/BostonHousing.csv')
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
import tensorflow as tf

In [4]:
def r2_metric(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))  # Sum of squares of residuals
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))  # Total sum of squares
    return 1 - SS_res / (SS_tot + tf.keras.backend.epsilon())  # R² score formula


In [5]:

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization

X = df.drop('medv', axis=1)
y = df['medv']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Crear modelo
model = Sequential()
model.add(BatchNormalization(input_shape=(X_train.shape[1],)))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

# Compilar y entrenar modelo
model.compile(metrics = [r2_metric],loss='mse', optimizer='adam')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
history = model.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_test, y_test), verbose=1)


Epoch 1/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 611.7994 - r2_metric: -8.4367 - val_loss: 553.7347 - val_r2_metric: -7.0907
Epoch 2/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 569.4363 - r2_metric: -6.2655 - val_loss: 541.2671 - val_r2_metric: -6.9050
Epoch 3/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 578.1673 - r2_metric: -6.8867 - val_loss: 526.5015 - val_r2_metric: -6.6813
Epoch 4/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 566.7723 - r2_metric: -11.5143 - val_loss: 510.0374 - val_r2_metric: -6.4313
Epoch 5/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 507.9959 - r2_metric: -5.1981 - val_loss: 490.6169 - val_r2_metric: -6.1379
Epoch 6/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 498.8860 - r2_metric: -7.3955 - val_loss: 465.2393 - val_r2_metric: -5.7502
Epoch 7/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 465.9912 - r2_metric: -23.8683 - val_loss: 434.5367 - val_r2_metric: -5.2831
Epoch 8/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

In [15]:
import plotly.graph_objects as go
import numpy as np


# Obtener el error de entrenamiento y validación de cada época
def plot_error(history):
  train_loss = history.history['loss']
  val_loss = history.history['val_loss']
  r2_train = history.history['r2_metric']
  r2_val = history.history['val_r2_metric']

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=np.arange(1, 301), y=train_loss, mode='lines', name='Error de entrenamiento'))
  fig.add_trace(go.Scatter(x=np.arange(1, 301), y=val_loss, mode='lines', name='Error de validación'))

  fig2 = go.Figure()
  fig2.add_trace(go.Scatter(x=np.arange(1, 301), y=r2_train, mode='lines', name='R2 de entrenamiento'))
  fig2.add_trace(go.Scatter(x=np.arange(1, 301), y=r2_val, mode='lines', name='R2 de validación'))

  fig.update_layout(
      title='Error de entrenamiento y validación',
      xaxis_title='Épocas',
      yaxis_title='Error (MSE)',
      legend_title='Leyenda'
  )
  fig2.update_layout(
      title='R2 de entrenamiento y validación',
      xaxis_title='Épocas',
      yaxis_title='R2',
      legend_title='Leyenda'
  )

  fig.show()
  fig2.show()

plot_error(history)

In [16]:
from keras.layers import Dropout
import plotly.graph_objects as go
import numpy as np

X = df.drop('medv', axis=1)
y = df['medv']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Crear modelo con Dropout
dropout_rate = 0.2  # Puedes ajustar este valor para encontrar la mejor tasa de dropout
model = Sequential()
model.add(BatchNormalization(input_shape=(X_train.shape[1],)))
model.add(Dense(10, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(10, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(10, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(1))

# Compilar y entrenar modelo
early_stopping = keras.callbacks.EarlyStopping(monitor='val_r2_metric', patience=10, restore_best_weights=True)
model.compile(metrics = [r2_metric],loss='mse', optimizer='adam')

history = model.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_test, y_test), verbose=1)
plot_error(history)


Epoch 1/300


/usr/local/lib/python3.10/dist-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - loss: 640.2783 - r2_metric: -6.7580 - val_loss: 440.2111 - val_r2_metric: -5.3969
Epoch 2/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 597.4207 - r2_metric: -6.6022 - val_loss: 444.9954 - val_r2_metric: -5.4642
Epoch 3/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 608.6363 - r2_metric: -6.4623 - val_loss: 440.5008 - val_r2_metric: -5.3937
Epoch 4/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 620.3051 - r2_metric: -6.8880 - val_loss: 432.6090 - val_r2_metric: -5.2730
Epoch 5/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 590.7463 - r2_metric: -6.0752 - val_loss: 421.0247 - val_r2_metric: -5.0983
Epoch 6/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 572.0613 - r2_metric: -13.2662 - val_loss: 406.1902 - val_r2_metric: -4.8754
Epoch 7/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 569.4210 - r2_metric: -34.0586 - val_loss: 386.7309 - val_r2_metric: -4.5837
Epoch 8/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 503.

In [17]:
from keras.regularizers import l1


X = df.drop('medv', axis=1)
y = df['medv']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)


l1_reg_strength = 0.01

model = Sequential()
model.add(BatchNormalization(input_shape=(X_train.shape[1],)))
model.add(Dense(10, activation='relu', kernel_regularizer=l1(l1_reg_strength)))
model.add(Dense(10, activation='relu', kernel_regularizer=l1(l1_reg_strength)))
model.add(Dense(10, activation='relu', kernel_regularizer=l1(l1_reg_strength)))
model.add(Dense(1))

# Compilar y entrenar modelo
early_stopping = keras.callbacks.EarlyStopping(monitor='val_r2_metric', patience=10, restore_best_weights=True, mode='max')
model.compile(metrics=[r2_metric], loss='mse', optimizer='adam')
history = model.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])


plot_error(history)


Epoch 1/300


/usr/local/lib/python3.10/dist-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 584.6414 - r2_metric: -18.6898 - val_loss: 577.7451 - val_r2_metric: -7.4384
Epoch 2/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 614.7947 - r2_metric: -6.8129 - val_loss: 568.2394 - val_r2_metric: -7.2891
Epoch 3/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 601.6952 - r2_metric: -42.8627 - val_loss: 558.4752 - val_r2_metric: -7.1381
Epoch 4/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 567.0944 - r2_metric: -21.0083 - val_loss: 545.7208 - val_r2_metric: -6.9423
Epoch 5/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 550.4861 - r2_metric: -12.6479 - val_loss: 532.6359 - val_r2_metric: -6.7411
Epoch 6/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 574.0951 - r2_metric: -15.5384 - val_loss: 518.2737 - val_r2_metric: -6.5183
Epoch 7/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 531.9894 - r2_metric: -6.5437 - val_loss: 501.3438 - val_r2_metric: -6.2569
Epoch 8/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 49

In [18]:

from keras import regularizers


X = df.drop('medv', axis=1)
y = df['medv']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

def train_model_with_l2(l2_lambda):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(X_train.shape[1],)))
    model.add(Dense(10, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)))
    model.add(Dense(10, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)))
    model.add(Dense(10, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)))
    model.add(Dense(1))

    model.compile(metrics=[r2_metric], loss='mse', optimizer='adam')
    history = model.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    return history

l2_lambdas = [0.001, 0.01, 0.1]
best_l2_lambda = None
best_val_r2 = -float('inf')

for l2_lambda in l2_lambdas:
    history = train_model_with_l2(l2_lambda)
    val_r2 = max(history.history['val_r2_metric'])
    if val_r2 > best_val_r2:
        best_val_r2 = val_r2
        best_l2_lambda = l2_lambda

print(f"El mejor valor de lambda para L2 es: {best_l2_lambda}")

best_history = train_model_with_l2(best_l2_lambda)


plot_error(best_history)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



El mejor valor de lambda para L2 es: 0.1


/usr/local/lib/python3.10/dist-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

